In [1]:
!pip install scikit-learn pandas datasets transformers torchvision evaluate accelerate

In [9]:
import torch
from utils import xception, TransferModel, shuffled_image_paths, create_path_label_dataset, get_transforms, evaluate_model, DeepFakeDetector

In [3]:
old_model = xception()
model = TransferModel(model=old_model,
                             num_out_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [4]:
folder_true = '/home/jovyan/shared/S25/nochange/true'
folder_false = '/home/jovyan/shared/S25/nochange/false'

transforms = get_transforms(old_model)
dataset = create_path_label_dataset(folder_true, folder_false)
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.with_transform(transforms)

labels = dataset['train'].features["label"].names

In [5]:
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer, DefaultDataCollator

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="deepfake-unchanged",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DefaultDataCollator(),
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [6]:
# trainer.train()
labels

['Real', 'Fake']

In [7]:
from safetensors.torch import load

from utils import xception_from_tensors
new_model = xception_from_tensors("deepfake-1/checkpoint-153/model.safetensors")

In [10]:
detector = DeepFakeDetector("deepfake-1/checkpoint-153/model.safetensors")
detector.detect('/home/jovyan/shared/S25/nochange/false/000_003_frame247.png')

{'real': 0.7077859044075012, 'fake': 0.292214035987854}

In [12]:
evaluate_model(model, dataset['test'], compute_metrics)

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


{'eval_loss': 0.6973223090171814,
 'eval_accuracy': 0.493,
 'eval_runtime': 56.9494,
 'eval_samples_per_second': 17.559,
 'eval_steps_per_second': 2.195}